In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count/ --recursive --human --summarize | tail -30

In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count_0430/").createOrReplaceTempView("ha")
spark.sql("select * from ha order by test_date desc").show(20, False)

In [0]:

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text = """

WITH store_db AS (
    SELECT
        db_table(
            engine="PG",
            database="AA_DAILY_EST",
            sql="select count(*) FROM store.store_est_fact_v2 WHERE granularity in ('daily') and date in ('2020-03-28') and est_free_app_download is not null "
        )
);

"""
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/"],
        }
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select * from store_db").show()

# 

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text ="""



WITH domain_id_name_mapping AS (
    SELECT
        db_table(
            engine="plproxy",
            database="aa",
            sql="SELECT domain_id AS domain_id, name AS domain_name FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0)"
        )
);

WITH store_db AS (
    SELECT
        db_table(
            engine="PG",
            database="AA_DAILY_EST",
            sql="select * from store.store_est_fact_v2 "
        )
);


"""



test_date='2020-03-24'
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(test_date)],
        }
    ]
}

run(spark, ingest_msg, sql_text)
spark.sql("select * from store_db").show()





In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text ="""

-- mapping feed as metrc in raw
WITH feed_metric AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);


-- select tested column from raw data
WITH metric_raw_data AS (
SELECT id, category_id as raw_category_id,rank,store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (3,4,5,6, 1002,1003, 1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_raw AS (
SELECT count(id) AS total_count , raw_category_id, raw_store_id, metric,device_code,date,platform from metric_raw_data where raw_store_id not in (3,4,5,6, 1002,1003, 1004, 1005, 1006,1007) group by raw_category_id, raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_rank_raw AS (

SELECT 
free_app_download,revenue, paid_app_download, raw_category_id,raw_store_id,device_code, platform,date
FROM
      group_by_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);




-- select tested column from raw data
WITH metric_raw_store_data AS (
SELECT distinct id, est, store_id as raw_store_id , metric,device_code, date , platform from feed_metric where store_id not in (3,4,5,6, 1002,1003, 1005,1004, 1006,1007)

);


-- group by and count data in raw data
WITH group_by_store_raw AS (
SELECT count(est) AS total_count ,raw_store_id, metric,device_code,date,platform from metric_raw_store_data where raw_store_id not in (3,4,5,6,1002,1003, 1004, 1005, 1006,1007) group by raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_store_raw AS (

SELECT 
free_app_download,revenue, paid_app_download,raw_store_id,device_code, platform, date
FROM
      group_by_store_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);



-- map raw with category
WITH category_mapping_raw AS (

SELECT * from ( select *, 'ios' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store' 
UNION ALL select *, 'android' as mapping_platform from category_mapping_deminsion_service where market_code='google-play'
 ) as mapping right join pivot_metric_rank_raw on mapping.legacy_category_id=pivot_metric_rank_raw.raw_category_id and 
mapping.mapping_platform=pivot_metric_rank_raw.platform
);


-- map raw with rank country_code
WITH country_category_mapping_rank_raw AS (
select date, raw_store_id, country_code,device_code,category_id,free_app_download,paid_app_download,revenue from country_code_mapping right join category_mapping_raw on country_code_mapping.country_code_store_id=category_mapping_raw.raw_store_id and country_code_mapping.market_code=category_mapping_raw.platform
);



-- map raw with store country_code
WITH country_mapping_store_raw AS (
select date, raw_store_id, country_code,device_code,free_app_download,paid_app_download,revenue from country_code_mapping right join pivot_metric_store_raw on country_code_mapping.country_code_store_id=pivot_metric_store_raw.raw_store_id and country_code_mapping.market_code=pivot_metric_store_raw.platform
);


-- group by unified data
WITH unified_group_data AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code, category_id as unified_category_id from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code, category_id from  rank_unified ) as unified
group by  category_id,  country_code,  device_code );


-- group by unified data
WITH unified_group_data_store AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code from  store_unified ) as unified
group by   country_code,  device_code );




-- compare raw vs unified data
WITH compared_data_rank AS (
    SELECT * from country_category_mapping_rank_raw left join unified_group_data on unified_group_data.unified_country_code==country_category_mapping_rank_raw.country_code and unified_group_data.unified_category_id==country_category_mapping_rank_raw.category_id and unified_group_data.unified_device_code==country_category_mapping_rank_raw.device_code
);

WITH miss_data_rank AS (
select * from compared_data_rank where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue or unified_count_app_id is null
)



-- compare raw vs unified data store
WITH compared_store_data AS (
    SELECT * from country_mapping_store_raw left join unified_group_data_store on unified_group_data_store.unified_country_code==country_mapping_store_raw.country_code and unified_group_data_store.unified_device_code==country_mapping_store_raw.device_code
);


WITH miss_data_store AS (
select * from compared_store_data where free_app_download!=unified_count_free_app_download or paid_app_download!=unified_count_paid_app_download or revenue!=unified_count_revenue or unified_count_app_id is null
)


"""



test_date='2020-03-01'
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(test_date)],
        },        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"rank_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(test_date)],
        },   
        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"store_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(test_date)],
        },
        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"category_mapping_deminsion_service",
            "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
        },   
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"ios_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
        },   
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"android_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
        }
    ]
}
d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",
                        sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                         "country_code").withColumn(
        "market_code", F.lit("ios"))
d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",
                        sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                         "country_code").withColumn(
        "market_code", F.lit("android"))

country_code_df = d1.union(d2).cache()
country_code_df = country_code_df.withColumnRenamed("store_id", "country_code_store_id")
print 'country mapping table'
country_code_df.show(2)
country_code_df.createOrReplaceTempView("country_code_mapping")

category_mapping_table = spark.read.parquet(
        "s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping")
category_mapping_table.createOrReplaceTempView("category_mapping_deminsion_service")

run(spark, ingest_msg, sql_text)
spark.sql("select * from store_db").show()
result1 = spark.sql("select * from miss_data_store")
result2 = spark.sql("select * from miss_data_rank")

result1.show()
result2.show()



# s3://b2c-prod-data-pipeline-qa/aa.store/result_store_v1


In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text = """

CREATE TABLE test_rank (
    country_code text NOT NULL,
    device_code int8 NOT NULL,
    category_id text NOT NULL,
    free_app_download text NOT NULL,
    paid_app_download text NOT NULL,
    revenue int4 NOT NULL,
    date date NOT NULL,
    PRIMARY KEY(country_code, device_code,category_id,free_app_download,paid_app_download,revenue)
)
PARTITION BY COLUMNS(date)
WITH (
    NAMESPACE = 'aa.store',
    COALESCE = 1,
    SAVEMODE = 'append',
    DIMENSIONS = COLUMNS(country_code, device_code,category_id),
    METRICS = COLUMNS(free_app_download,paid_app_download,revenue)
);



-- mapping feed as metrc in raw
WITH feed_metric AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);




-- select tested column from raw data
WITH metric_raw_data AS (
SELECT id, category_id as raw_category_id,rank, store_id as raw_store_id , metric,device_code,date , platform from feed_metric
);


-- group by and count data in raw data
WITH group_by_raw AS (
SELECT count(id) AS total_count , raw_category_id, raw_store_id, metric,device_code,date,platform from metric_raw_data where raw_store_id not in (0,3,4,5,6, 1000, 1003, 1005, 1006,1007)  group by raw_category_id, raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_raw AS (

SELECT 
free_app_download,revenue, paid_app_download, raw_category_id,raw_store_id,device_code, platform, date
FROM
      group_by_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download', 'revenue', 'paid_app_download')
  )
);



-- map raw with category
WITH category_mapping_raw AS (

SELECT * from ( select *, 'ios' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store' 
UNION ALL select *, 'android' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store'
 ) as mapping full outer join pivot_metric_raw on mapping.legacy_category_id=pivot_metric_raw.raw_category_id and 
mapping.mapping_platform=pivot_metric_raw.platform
);



-- union all platform with country_code mapping

WITH country_code_mapping AS (
select *, 'android' as market_code from android_country_mapping 
UNION ALL select *, 'ios' market_code from ios_country_mapping
);



-- map raw with country_code

WITH country_category_mapping_raw AS (
select date, country_code,device_code,category_id,free_app_download,paid_app_download,revenue from country_code_mapping full outer join category_mapping_raw on country_code_mapping.store_id=category_mapping_raw.raw_store_id and country_code_mapping.market_code=category_mapping_raw.platform
);

-- group by unified data
WITH unified_group_data AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code, category_id as unified_category_id from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code, category_id from  rank_unified ) as unified
group by  category_id,  country_code,  device_code );



-- compare raw vs unified data
WITH compared_data AS (
    SELECT * from country_category_mapping_raw left join unified_group_data on unified_group_data.unified_country_code==country_category_mapping_raw.country_code and unified_group_data.unified_category_id==country_category_mapping_raw.category_id and unified_group_data.unified_device_code==country_category_mapping_raw.device_code
);

WITH miss_data AS (
select * from compared_data where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue or unified_count_app_id is null
)

"""
# PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
# set search_path=store;

# select * from store.store_est_fact_v2 where country_code='US' and app_id='835599320' and granularity='daily' and date='2020-03-28' ;

# EOF

# WITH domain_id_name_mapping AS (
#     SELECT
#         db_table(
#             engine="plproxy",
#             database="aa",
#             sql="SELECT domain_id AS domain_id, name AS domain_name FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0)"
#         )
# );



# WITH store_db AS (
#     SELECT
#         db_table(
#             engine="CITUS",
#             database="AA_CITUS",
#             sql="select * from store.store_est_fact_v2 "
#         )
# );


test_date='2020-03-24'
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(test_date)],
        },        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"rank_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(test_date)],
        },   
        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"store_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(test_date)],
        },
        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"category_mapping_deminsion_service",
            "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
        },   
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"ios_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
        },   
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"android_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
        }
    ]
}
run(spark, ingest_msg, sql_text)
result = spark.sql("select * from miss_data")


result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_v1/",mode="append",
            partitionBy=["date"])


# s3://b2c-prod-data-pipeline-qa/aa.store/result_store_v1


In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text = """
-- mapping feed as metrc in raw
WITH feed_metric_store AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);


-- select tested column from raw data
WITH metric_raw_store_data AS (
SELECT distinct id, est, store_id as raw_store_id , metric,device_code,date , platform from feed_metric_store
);


-- group by and count data in raw data
WITH group_by_store_raw AS (
SELECT count(est) AS total_count ,raw_store_id, metric,device_code,date,platform from metric_raw_store_data where raw_store_id not in (0,3,4,5,6, 1000, 1003, 1005, 1006,1007) group by raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_store_raw AS (

SELECT 
free_app_download,revenue, paid_app_download,raw_store_id,device_code, platform
FROM
      group_by_store_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);


-- union all platform with country_code mapping

WITH country_code_mapping AS (
select *, 'android' as market_code from android_country_mapping 
UNION ALL select *, 'ios' market_code from ios_country_mapping
);



-- map raw with country_code

WITH country_mapping_store_raw AS (
select country_code,device_code,free_app_download,paid_app_download,revenue from country_code_mapping full outer join pivot_metric_store_raw on country_code_mapping.store_id=pivot_metric_store_raw.raw_store_id and country_code_mapping.market_code=pivot_metric_store_raw.platform
);



-- group by unified data
WITH unified_group_data_store AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code, device_code  from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code from  store_unified ) as unified
group by   country_code,  device_code );



-- compare raw vs unified data
WITH compared_store_data AS (
    SELECT * from country_mapping_store_raw left join unified_group_data_store on unified_group_data_store.country_code==country_mapping_store_raw.country_code and unified_group_data_store.device_code==country_mapping_store_raw.device_code
);


WITH miss_data AS (
select * from compared_store_data where free_app_download!=unified_count_free_app_download or paid_app_download!=unified_count_paid_app_download or revenue!=unified_count_revenue or unified_count_app_id is null
)


"""


# store_unified , rank_unified
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "name":"rank_raw",
            "data_encoding": "parquet",
            "compression": "gzip",
            "path": ["s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-03-28/"],
        },        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"store_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-03-28"],
        },        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"category_mapping_deminsion_service",
            "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
        }, 
        {
            "data_encoding": "parquet",
            "compression": "gzip",
          "name":"rank_unified",
            "path": ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2020-03-28"],
        },
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"ios_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
        },   
        {
            "data_encoding": "csv",
            "compression": "gzip",
            "name":"android_country_mapping",
            "data_schema": [
                            {"name":"store_id","type":"int","nullable": False},
                            {"name":"country_code","type":"string","nullable": False},
                            {"name":"country_name","type":"string","nullable": False}
                            ],
             "csv_options": {
           'header': True,
           'sep': '\t',
           'quote': '',
           'encoding': 'utf-8',
           'escape': ''
           },

            "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
        }
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select * from miss_data ").show()

# spark.sql("select * from compared_data where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue").show(2)

# spark.sql("select * from store_db").show(2)





In [0]:

import datetime

start = "2020-01-01"
end = "2020-01-03"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

est_list = [ "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(test_date) for test_date in dates ]
rank_list = ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(test_date) for test_date in dates  ]
print est_list
print rank_list

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
import datetime

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


start='2010-07-04'
end='2013-01-01'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.reverse()


def test_count(test_date):
    # test_date = '2017-08-01'
    namespace = "aa.store.market-size.v1"
    
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "category_mapping_deminsion_service",
                "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
            },
            {
                "data_encoding": "csv",
                "compression": "gzip",
                "name": "ios_country_mapping",
                "data_schema": [
                    {"name": "store_id", "type": "int", "nullable": False},
                    {"name": "country_code", "type": "string", "nullable": False},
                    {"name": "country_name", "type": "string", "nullable": False}
                ],
                "csv_options": {
                    'header': True,
                    'sep': '\t',
                    'quote': '',
                    'encoding': 'utf-8',
                    'escape': ''
                },
    
                "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
            },
            {
                "data_encoding": "csv",
                "compression": "gzip",
                "name": "android_country_mapping",
                "data_schema": [
                    {"name": "store_id", "type": "int", "nullable": False},
                    {"name": "country_code", "type": "string", "nullable": False},
                    {"name": "country_name", "type": "string", "nullable": False}
                ],
                "csv_options": {
                    'header': True,
                    'sep': '\t',
                    'quote': '',
                    'encoding': 'utf-8',
                    'escape': ''
                },
    
                "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
            }
        ]
    }
    
    sql_text = """
    
    
    -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'
    );
    
    
    
           WITH unified_category_filter_data_free_app_download AS 
    ( 
           SELECT * ,
           CASE WHEN (free_app_download > 1000 and country_code !='WW') or (free_app_download > 4000 and country_code =='WW' ) or (free_app_download is null or free_app_download <= 0) Then null else est_free_app_download END as est_free_app_download_category,
           CASE WHEN (paid_app_download > 1000 and country_code !='WW') or (paid_app_download > 4000 and country_code =='WW' ) or (paid_app_download is null or paid_app_download <= 0) Then null else est_paid_app_download END as est_paid_app_download_category,
           CASE WHEN (revenue > 1000 and country_code !='WW') or (revenue > 4000 and country_code =='WW') or (revenue is null or revenue <= 0) Then null else est_revenue  END as est_revenue_category
           FROM   unified_rank_filter_data_free_app_download 


    );    

    
    """


    run(spark, ingest_msg, sql_text)
    # current est test:
    free_app_download_count = spark.sql(
        "select count(app_id) from (select distinct app_id, country_code, device_code, est_free_app_download from unified_rank_filter_data_free_app_download where est_free_app_download is not null ) as test").take(
        1)
    paid_app_download_count = spark.sql(
        "select count(app_id) from (select distinct app_id, country_code, device_code, est_paid_app_download from unified_rank_filter_data_free_app_download where est_paid_app_download is not null ) as test").take(
        1)
    revenue_count = spark.sql(
        "select count(app_id) from (select distinct app_id, country_code, device_code, est_revenue from unified_rank_filter_data_free_app_download where est_revenue is not null ) as test ").take(
        1)
        
        

    free_app_download_category_count = spark.sql(
        "select count(*) from (select app_id, country_code, device_code, category_id, est_free_app_download from unified_category_filter_data_free_app_download where est_free_app_download_category is not null ) as test").take(
        1)
    paid_app_download_category_count = spark.sql(
        "select count(*) from (select app_id, country_code, device_code, category_id, est_paid_app_download from unified_category_filter_data_free_app_download where est_paid_app_download_category is not null ) as test").take(
        1)
    revenue_category_count = spark.sql(
        "select count(*) from (select app_id, country_code, device_code, category_id, est_revenue from unified_category_filter_data_free_app_download where est_revenue_category is not null ) as test ").take(
        1)
    
    import psycopg2
    import datetime
    
    spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
    import aaplproxy
    from aadatapipelinecore.core.urn import Urn
    from aaplproxy.da.local_sqlrunner import LocalSqlRunner
    from aadatapipelinecore.core.utils.module import application_settings
    from pyspark.sql import Row
    
    
    def citus_row_category(date, country_code, market_code, device_code, category_id, metric_name):
        def get_data_in_citus(date, country_code, market_code, device_code, category_id, metric_name):
            citus_dsn_ = (
                "dbname='{db}' user='{user}' password='{password}' "
                "host='{host}' port='{port}'".format(
                    db="aa_store_db",
                    user="citus_bdp_prod_app_int_qa",
                    host="10.2.10.254",
                    password="wZw8cfBuuklIskVG",
                    port=5432
                )
            )
            # sql_category = "select * from ( select app_id,country_code,device_code,category_id, date,{} as estimate from  store.store_est_category_fact_v1 where granularity='daily' and date='{}' and device_code='{}' and {} is not null and country_code='{}' and category_id={} ) as category_count ".format(metric_name, date, device_code, metric_name, country_code, category_id )
            # sql_est = "select * from ( select app_id,country_code,device_code,date,{} as estimate from  store.store_est_fact_v3 where granularity='daily' and date='{}' and device_code='{}' and {} is not null and country_code='{}' ) as est_count ".format(metric_name, date, device_code, metric_name, country_code )
            # sql_category="select * from store.store_est_category_fact_v1 limit 1"
            # sql_est = "select app_id from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null and country_code='WW' and device_code='ios-phone' ) as prod "
            # sql_est = "select app_id from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null ) as prod "
            sql_est_free = "select count(*) from store.store_est_fact_v6 where date='{}' and granularity='daily' and est_free_app_download is not null ".format(
                date)
            sql_est_paid = "select count(*) from store.store_est_fact_v6 where date='{}' and granularity='daily' and est_paid_app_download is not null ".format(
                date)
            sql_est_download = "select count(*) from store.store_est_fact_v6 where date='{}' and granularity='daily' and est_revenue is not null ".format(
                date)
                
            sql_category_free = "select count(*) from store.store_est_category_fact_v6 where date='{}' and granularity='daily' and est_free_app_download is not null".format(
                date)
            sql_category_paid = "select count(*) from store.store_est_category_fact_v6 where date='{}' and granularity='daily' and est_paid_app_download is not null".format(
                date)
            sql_category_revenue = "select count(*) from store.store_est_category_fact_v6 where date='{}' and granularity='daily' and est_revenue is not null".format(
                date)
                
            sql_est_app_count = "select count (distinct app_id) from store.store_est_fact_v6 where date='{}' and granularity='daily' ".format(date) ; 
            sql_category_app_count = "select count (distinct app_id) from store.store_est_category_fact_v6 where date='{}' and granularity='daily' ".format(date) ; 


            sql_est_app_count = "select count (distinct app_id) from store.store_est_fact_v6 where date='{}' and granularity='daily' ".format(date) ; 
            sql_category_app_count = "select count (distinct app_id) from store.store_est_category_fact_v6 where date='{}' and granularity='daily' ".format(date) ; 



            # db_category_count_result = ''  # query(citus_dsn_, sql_category)
    
            query_list = [sql_est_free, sql_est_paid, sql_est_download]
            data_est_count_result = [query(citus_dsn_, sql) for sql in query_list]
            
            query_list_category = [sql_category_free, sql_category_paid, sql_category_revenue]
            data_category_count_result = [query(citus_dsn_, sql) for sql in query_list_category]

            data_est_app_id_count_result = query(citus_dsn_, sql_est_app_count)
            data_category_app_id_count_result = query(citus_dsn_, sql_category_app_count)

    
            # print 'running.....'
            return data_category_count_result, data_est_count_result, data_est_app_id_count_result, data_category_app_id_count_result
    
        def query(dsn, sql):
            with psycopg2.connect(dsn) as conn:
                conn.autocommit = True
                with conn.cursor() as cur:
                    cur.execute(sql)
                    result = cur.fetchall()
                    conn.commit()
            return result
    
        result_category, result_est, result_est_app_count, result_category_app_count = get_data_in_citus(date, country_code, market_code, device_code, category_id,
                                                        metric_name)
        return  [ Row(app_id=r[0]) for r in result_category], [Row(app_id=r[0]) for r in result_est], [Row(app_id=result_est_app_count[0])], [Row(app_id=result_category_app_count[0])]
        # return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], date=r[3], estimate=r[4]) for r in result_est ] , len(result)
    
    
    db_test_result_category, db_test_result_est, db_est_count, db_category_count = citus_row_category(test_date, 'US', 'apple-store', 'ios-phone', 100000,
                                        'est_free_app_download')
    
    unified_result = [free_app_download_count[0][0], paid_app_download_count[0][0], revenue_count[0][0]]
    db_result = [compare_data[0][0] for compare_data in db_test_result_est]
    
    unified_category_result=[free_app_download_category_count[0][0], paid_app_download_category_count[0][0], revenue_category_count[0][0]]
    db_category_result = [compare_data[0][0] for compare_data in db_test_result_category]
    
    # print unified_category_result
    # print db_category_result
    from datetime import datetime
    df_write_result = spark.createDataFrame([('est', test_date.strftime("%Y-%m-%d"), unified_result, db_result), 
                                            ("category", test_date.strftime("%Y-%m-%d"), unified_category_result, db_category_result)], 
                                            schema=["type","test_date","unified","db"])

    from aadatapipelinecore.core.utils.retry import retry
    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count_0430/",
                                          mode="append",
                                          partitionBy=["type"])
    retry(write_test_result,(df_write_result,),{},interval=10)
    
    
    if unified_result == db_result:
        print test_date, 'est pass'
    else:
        print test_date, 'est failed!!!!!'


    if unified_category_result == db_category_result:
        print test_date, 'category pass'
    else:
        print test_date, 'category failed!!!!!'


    if db_est_count[0][0] == db_category_count[0][0]:
        print test_date, 'app_count pass'
    else:
        print test_date, 'app_count failed!!!!!'



sc.parallelize(map(test_count, dates), 1)

# for x in dates:
#     print x
#     test_count(x)


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count_0430/ | sort -n

In [0]:
/

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_unified_db_count_0430/part-00004-7e709799-1b0f-4d4a-a301-cb218ade0f92-c000.snappy.parquet").show()


In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
import datetime

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


start='2020-03-02'
end='2020-04-01'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))


def test_count(test_date):
    # test_date = '2017-08-01'
    namespace = "aa.store.market-size.v1"
    
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "category_mapping_deminsion_service",
                "path": ["s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping"],
            },
            {
                "data_encoding": "csv",
                "compression": "gzip",
                "name": "ios_country_mapping",
                "data_schema": [
                    {"name": "store_id", "type": "int", "nullable": False},
                    {"name": "country_code", "type": "string", "nullable": False},
                    {"name": "country_name", "type": "string", "nullable": False}
                ],
                "csv_options": {
                    'header': True,
                    'sep': '\t',
                    'quote': '',
                    'encoding': 'utf-8',
                    'escape': ''
                },
    
                "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING"],
            },
            {
                "data_encoding": "csv",
                "compression": "gzip",
                "name": "android_country_mapping",
                "data_schema": [
                    {"name": "store_id", "type": "int", "nullable": False},
                    {"name": "country_code", "type": "string", "nullable": False},
                    {"name": "country_name", "type": "string", "nullable": False}
                ],
                "csv_options": {
                    'header': True,
                    'sep': '\t',
                    'quote': '',
                    'encoding': 'utf-8',
                    'escape': ''
                },
    
                "path": ["s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING"],
            }
        ]
    }
    
    sql_text = """
    
    
    -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'

    );
    
    
       WITH unified_category_filter_data_free_app_download AS 
    ( 
           SELECT * ,
           CASE WHEN (free_app_download > 1000 and country_code !='WW') or (free_app_download > 4000 and country_code =='WW' ) Then null else est_free_app_download END as est_free_app_download_category,
           CASE WHEN (paid_app_download > 1000 and country_code !='WW') or (paid_app_download > 4000 and country_code =='WW' ) Then null else est_paid_app_download END as est_paid_app_download_category,
           CASE WHEN (revenue > 1000 and country_code !='WW') or (revenue > 4000 and country_code =='WW') Then null else est_revenue  END as est_revenue_category
           FROM   unified_rank_filter_data_free_app_download 


    );    

    
    
    
    """
    
#     [9049417, 1914843, 5070231]
# [8620540L, 1901239L, 4578835L]

# [9049417, 1914843, 5070231]
# [8620540L, 1901239L, 4578835L]


    run(spark, ingest_msg, sql_text)
 
        
    # spark.sql(
    #     "select * from (select app_id, country_code, device_code, category_id, est_free_app_download from unified_rank_filter_data_free_app_download where est_free_app_download is not null ) as test").show()
    # free_app_download_category_count = spark.sql(
    #     "select * from (select app_id, country_code, device_code, category_id, est_free_app_download from unified_rank_filter_data_free_app_download where est_free_app_download is not null and category_id=100000 ) as test").show(2000000)
        
    free_app_download_category_count = spark.sql(
        "select * from unified_category_filter_data_free_app_download where paid_app_download is null   ").show()

    # paid_app_download_category_count = spark.sql(
    #     "select count(*) from (select app_id, country_code, device_code, category_id, est_paid_app_download from unified_rank_filter_data_free_app_download where est_paid_app_download is not null ) as test").take(
    #     1)
    # revenue_category_count = spark.sql(
    #     "select count(*) from (select app_id, country_code, device_code, category_id, est_revenue from unified_rank_filter_data_free_app_download where est_revenue is not null ) as test ").take(
    #     1)
    
    # import psycopg2
    # import datetime
    
    # spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
    # import aaplproxy
    # from aadatapipelinecore.core.urn import Urn
    # from aaplproxy.da.local_sqlrunner import LocalSqlRunner
    # from aadatapipelinecore.core.utils.module import application_settings
    # from pyspark.sql import Row
    
    
    # def citus_row_category(date, country_code, market_code, device_code, category_id, metric_name):
    #     def get_data_in_citus(date, country_code, market_code, device_code, category_id, metric_name):
    #         citus_dsn_ = (
    #             "dbname='{db}' user='{user}' password='{password}' "
    #             "host='{host}' port='{port}'".format(
    #                 db="aa_store_db",
    #                 user="citus_bdp_prod_app_int_qa",
    #                 host="10.2.10.254",
    #                 password="wZw8cfBuuklIskVG",
    #                 port=5432
    #             )
    #         )

    #         sql_category_free = "select count(*) from store.store_est_category_fact_v1 where date='{}' and granularity='daily' and est_free_app_download  is not null and country_code='AE' and device_code='ios-phone' and category_id=100000".format(
    #             date)
    #         # sql_category_paid = "select count(*) from store.store_est_category_fact_v1 where date='{}' and granularity='daily' and est_paid_app_download is not null and country_code='AE' and device_code='ios-phone'".format(
    #         #     date)
    #         # sql_category_revenue = "select count(*) from store.store_est_category_fact_v1 where date='{}' and granularity='daily' and est_revenue is not null and country_code='AE' and device_code='ios-phone'".format(
    #         #     date)

    #         # db_category_count_result = ''  # query(citus_dsn_, sql_category)
    
    #         # query_list = [sql_est_free, sql_est_paid, sql_est_download]
    #         # data_est_count_result = [query(citus_dsn_, sql) for sql in query_list]
            
    #         # query_list_category = [sql_category_free, sql_category_paid, sql_category_revenue]
    #         # data_category_count_result = [query(citus_dsn_, sql) for sql in query_list_category]
    #         query_list_category = [sql_category_free]
    #         data_category_count_result = [query(citus_dsn_, sql) for sql in query_list_category]

    
    #         print 'running.....'
    #         return data_category_count_result, []
    
    #     def query(dsn, sql):
    #         with psycopg2.connect(dsn) as conn:
    #             conn.autocommit = True
    #             with conn.cursor() as cur:
    #                 cur.execute(sql)
    #                 result = cur.fetchall()
    #                 conn.commit()
    #         return result
    
    #     result_category, result_est = get_data_in_citus(date, country_code, market_code, device_code, category_id,
    #                                                     metric_name)
    #     return  [ Row(app_id=r[0]) for r in result_category], [""]
    #     # return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], date=r[3], estimate=r[4]) for r in result_est ] , len(result)
    
    
    # db_test_result_category, db_test_result_est = citus_row_category(test_date, 'US', 'apple-store', 'ios-phone', 100000,
    #                                     'est_free_app_download')
    
    # #unified_result = [free_app_download_count[0][0], paid_app_download_count[0][0], revenue_count[0][0]]
    # #db_result = [compare_data[0][0] for compare_data in db_test_result_est]
    
    # unified_category_result=[free_app_download_category_count[0][0]]#, paid_app_download_category_count[0][0], revenue_category_count[0][0]]
    # db_category_result = [compare_data[0][0] for compare_data in db_test_result_category]
    # print unified_category_result
    # print db_category_result
    # # from datetime import datetime
    # # df_write_result = spark.createDataFrame([(test_date.strftime("%Y-%m-%d"), unified_result, db_result)], schema=["test_date","unified","db"])

    # # from aadatapipelinecore.core.utils.retry import retry
    # # def write_test_result(df_write_result):
    # #     df_write_result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_est_unified_db_count/",
    # #                                       mode="append",
    # #                                       partitionBy=["test_date"])
    # # retry(write_test_result,(df_write_result,),{},interval=10)
    # # if unified_result == db_result:
    # #     print test_date, 'est pass'
    # # else:
    # #     print test_date, 'est failed'


    # if unified_category_result == db_category_result:
    #     print test_date, 'category pass'
    # else:
    #     print test_date, 'category failed'





for x in dates:
    test_count(x)


# # spark.sql("select * from unified where app_id='1410134706' ").show()
# # spark.sql("select * from db where app_id='1109868900' ").show()

# spark.sql("select count(*) from db").show(2)
# spark.sql("select count(*) from unified_1").show(2)



    # def compared_category():
    #     spark.sql("select * from (select app_id, country_code, device_code, category_id, est_free_app_download from unified_data_with_filter where est_free_app_download is not null ) as test").show()
    #     spark.sql("select count(app_id) from (select app_id, country_code, device_code, category_id, est_free_app_download from unified_data_with_filter where est_free_app_download is not null ) as test").show()
    #     df_unified = spark.sql("select distinct app_id from (select distinct app_id, country_code, device_code, free_app_download_count from unified_rank_filter_data_free_app_download where free_app_download_count is not null ) as test").cache()
    #     df_unified.createOrReplaceTempView("unified_1")

    #     spark.sql("select app_id from db except all select app_id from unified_1").show()
    #     spark.sql("select app_id from unified_1 except all select app_id from db").show()

# SET
#   count  
# ---------
#  9729053
# (1 row)




In [0]:

IOS_STORE_COUNTRY_MAPPING = [
    (0, 'WW'), (143575, 'AL'), (143563, 'DZ'), (143564, 'AO'), (143538, 'AI'),
    (143540, 'AG'), (143505, 'AR'), (143524, 'AM'), (143460, 'AU'), (143445, 'AT'),
    (143568, 'AZ'), (143539, 'BS'), (143559, 'BH'), (143541, 'BB'), (143565, 'BY'),
    (143446, 'BE'), (143555, 'BZ'), (143576, 'BJ'), (143542, 'BM'), (143577, 'BT'),
    (143556, 'BO'), (143525, 'BW'), (143503, 'BR'), (143543, 'VG'), (143560, 'BN'),
    (143526, 'BG'), (143578, 'BF'), (143579, 'KH'), (143455, 'CA'), (143580, 'CV'),
    (143544, 'KY'), (143581, 'TD'), (143483, 'CL'), (143465, 'CN'), (143501, 'CO'),
    (143582, 'CG'), (143495, 'CR'), (143494, 'HR'), (143557, 'CY'), (143489, 'CZ'),
    (143458, 'DK'), (143545, 'DM'), (143508, 'DO'), (143509, 'EC'), (143516, 'EG'),
    (143506, 'SV'), (143518, 'EE'), (143583, 'FJ'), (143447, 'FI'), (143442, 'FR'),
    (143584, 'GM'), (143443, 'DE'), (143573, 'GH'), (143448, 'GR'), (143546, 'GD'),
    (143504, 'GT'), (143585, 'GW'), (143553, 'GY'), (143510, 'HN'), (143463, 'HK'),
    (143482, 'HU'), (143558, 'IS'), (143467, 'IN'), (143476, 'ID'), (143449, 'IE'),
    (143491, 'IL'), (143450, 'IT'), (143511, 'JM'), (143462, 'JP'), (143528, 'JO'),
    (143517, 'KZ'), (143529, 'KE'), (143493, 'KW'), (143586, 'KG'), (143587, 'LA'),
    (143519, 'LV'), (143497, 'LB'), (143588, 'LR'), (143520, 'LT'), (143451, 'LU'),
    (143515, 'MO'), (143530, 'MK'), (143531, 'MG'), (143589, 'MW'), (143473, 'MY'),
    (143532, 'ML'), (143521, 'MT'), (143590, 'MR'), (143533, 'MU'), (143468, 'MX'),
    (143591, 'FM'), (143523, 'MD'), (143592, 'MN'), (143547, 'MS'), (143593, 'MZ'),
    (143594, 'NA'), (143484, 'NP'), (143452, 'NL'), (143461, 'NZ'), (143512, 'NI'),
    (143534, 'NE'), (143561, 'NG'), (143457, 'NO'), (143562, 'OM'), (143477, 'PK'),
    (143595, 'PW'), (143485, 'PA'), (143597, 'PG'), (143513, 'PY'), (143507, 'PE'),
    (143474, 'PH'), (143478, 'PL'), (143453, 'PT'), (143498, 'QA'), (143487, 'RO'),
    (143469, 'RU'), (143598, 'ST'), (143479, 'SA'), (143535, 'SN'), (143599, 'SC'),
    (143600, 'SL'), (143464, 'SG'), (143496, 'SK'), (143499, 'SI'), (143601, 'SB'),
    (143472, 'ZA'), (143466, 'KR'), (143454, 'ES'), (143486, 'LK'), (143548, 'KN'),
    (143549, 'LC'), (143550, 'VC'), (143554, 'SR'), (143602, 'SZ'), (143456, 'SE'),
    (143459, 'CH'), (143470, 'TW'), (143603, 'TJ'), (143572, 'TZ'), (143475, 'TH'),
    (143551, 'TT'), (143536, 'TN'), (143480, 'TR'), (143604, 'TM'), (143552, 'TC'),
    (143537, 'UG'), (143492, 'UA'), (143481, 'AE'), (143444, 'GB'), (143441, 'US'),
    (143514, 'UY'), (143566, 'UZ'), (143502, 'VE'), (143471, 'VN'), (143571, 'YE'),
    (143605, 'ZW'),(143518, 'EE')]
ANDROID_STORE_COUNTRY_MAPPING = [
    (17, 'AR'), (1, 'AU'), (35, 'AT'), (61, 'AZ'), (11, 'BE'), (18, 'BR'), (47, 'BG'),
    (2, 'CA'), (13, 'CL'), (3, 'CN'), (52, 'CO'), (64, 'CR'), (80, 'HR'), (36, 'CZ'),
     (38, 'DK'), (62, 'EC'), (33, 'EG'), (20, 'FI'), (6, 'FR'), (4, 'DE'), (46, 'GR'),
   (16, 'HK'), (37, 'HU'), (19, 'IN'), (21, 'ID'), (39, 'IE'), (40, 'IL'), (8, 'IT'),
   (9, 'JP'), (53, 'KZ'), (95, 'KE'), (50, 'KW'), (86, 'LV'), (65, 'LB'), (78, 'LT'),
   (24, 'MY'), (26, 'MX'), (23, 'NL'), (41, 'NZ'), (74, 'NG'), (42, 'NO'), (54, 'PK'),
     (56, 'PE'), (31, 'PH'), (28, 'PL'), (43, 'PT'), (84, 'PR'), (73, 'QA'), (44, 'RO'),
    (22, 'RU'), (51, 'SA'), (32, 'SG'), (45, 'SK'), (14, 'ZA'), (27, 'KR'), (5, 'ES'),
     (34, 'SE'), (12, 'CH'), (30, 'TW'), (29, 'TH'), (25, 'TR'), (48, 'UA'), (49, 'AE'),
     (7, 'GB'), (10, 'US'), (15, 'VN'), (1000, 'WW')
 ]

est_list=["est_free_app_download", "est_paid_app_download", "est_revenue" ]
device_code_list=['ios-phone','ios-tablet']
date_list=['2019-08-01']
l1 = [[d, x[1], 'apple-store' , device_code, est] for x in IOS_STORE_COUNTRY_MAPPING for est in est_list for device_code in device_code_list for d in date_list]
l2 = [ [d, x[1], 'google-play' , "android-all", est] for x in ANDROID_STORE_COUNTRY_MAPPING for est in est_list for d in date_list]

test_data_list = l1 + l2

print test_data_list

In [0]:

%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.10.254  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

--\d+

--select count(*) from store.store_est_fact_v1 where date='2017-08-01' and granularity='daily' and est_free_app_download is not null   ; 
--select count(*) from store.store_est_fact_v1 where date='2017-08-01' and granularity='daily' and est_paid_app_download is not null  ; 
--select count(*) from store.store_est_fact_v1 where date='2017-08-01' and granularity='daily' and est_revenue is not null ; 
-- select * from store.store_est_category_fact_v1 where date='2020-03-02' and granularity='daily' and device_code='ios-phone' and country_code='AE' and app_id=523365306 and  category_id=100000 limit 3 ; 

--select count (distinct app_id) from store.store_est_fact_v1 where date='2020-02-01' and granularity='weekly'   limit 3 ; 


-- select count (distinct date) from store.store_est_fact_v1 limit 5
--select * from pg_stat_activity where query is not null limit 10;
-- select distinct date from store_est_category_fact_v1 order by date asc limit 10
-- select * from store.store_est_fact_v1 where date='2019-08-01' and est_free_app_download is not null limit 3;
-- select country_code, count(*) as count_country from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null  and device_code='ios-phone') as prod  group by country_code order by count_country asc

-- select app_id from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null  and device_code='ios-phone') as prod  group by country_code order by count_country asc
--select count(*) from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null and country_code='GD' and device_code='ios-phone'  ) as prod;

--select count(*) from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null  ) as prod;

-- select count(*) from ( select distinct app_id, country_code,device_code, est_paid_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_paid_app_download is not null  ) as prod;
-- select count(*) from ( select distinct app_id, country_code,device_code, est_revenue from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_revenue is not null  ) as prod;

--select * from store.store_est_fact_v4 where device_code='' limit 10  ;
--select * from store.store_est_category_fact_v4 where country_code='US' and category_id='100028' and app_id='966030322' and  granularity = 'daily' and date='2020-01-02';
-- \d+

explain (analyze true,buffers true)
select * from store.store_est_category_fact_v6 order by date desc limit 3 ; 


EOF


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row

def citus_row_category(date, country_code , market_code , device_code, category_id, metric_name):
    def get_data_in_citus(date, country_code , market_code , device_code, category_id, metric_name):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.10.254",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        # sql_category = "select * from ( select app_id,country_code,device_code,category_id, date,{} as estimate from  store.store_est_category_fact_v1 where granularity='daily' and date='{}' and device_code='{}' and {} is not null and country_code='{}' and category_id={} ) as category_count ".format(metric_name, date, device_code, metric_name, country_code, category_id )
        # sql_est = "select * from ( select app_id,country_code,device_code,date,{} as estimate from  store.store_est_fact_v3 where granularity='daily' and date='{}' and device_code='{}' and {} is not null and country_code='{}' ) as est_count ".format(metric_name, date, device_code, metric_name, country_code )
        # sql_category="select * from store.store_est_category_fact_v1 limit 1"
        # sql_est = "select app_id from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null and country_code='WW' and device_code='ios-phone' ) as prod "
        # sql_est = "select app_id from ( select distinct app_id, country_code,device_code, est_free_app_download from store.store_est_fact_v1  where date='2019-08-01' and granularity='daily' and est_free_app_download is not null ) as prod "
        # sql_est_free = "select count(*) from store.store_est_fact_v1 where date='{}' and granularity='daily' and est_free_app_download is not null ".format(date)
        # sql_est_paid = "select count(*) from store.store_est_fact_v1 where date='{}' and granularity='daily' and est_paid_app_download is not null ".format(date)
        # sql_est_download = "select count(*) from store.store_est_fact_v1 where date='{}' and granularity='daily' and est_revenue is not null ".format(date)

        db_category_count_result = '' #query(citus_dsn_, sql_category)
        
        query_list=[sql_est_free, sql_est_paid, sql_est_download]
        data_est_count_result = [ query(citus_dsn_, sql) for sql in query_list ]
        
        print 'running.....'
        return db_category_count_result , data_est_count_result

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result_category, result_est = get_data_in_citus(date, country_code , market_code , device_code, category_id, metric_name)
    return [Row(app_id=r[0]) for r in result_est]
    # return [ Row(app_id=r[0], country_code=r[1], device_code=r[2], date=r[3], estimate=r[4]) for r in result_est ] , len(result)

db_test_result = citus_row_category('2018-01-01', 'US' , 'apple-store' , 'ios-phone', 100000, 'est_free_app_download')



In [0]:

from aadatapipelinecore.core.utils.spark import eject_all_caches

eject_all_caches(spark)


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v2_2019-07-14/ | wc -l
